<a href="https://colab.research.google.com/github/JinyongShin/TensorFlow/blob/main/06_Softmax_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(777)

2.4.1


In [4]:
x_data = [[1, 2, 1,1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

y_data = [[0, 0, 1], 
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [6]:
nb_classes = 3 # number of classes

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [17]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')

variables = [W , b]
print(W , b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.69555974, -0.25555828,  0.04393371],
       [-1.3831443 ,  0.9659563 , -1.1959723 ],
       [-0.44151336, -0.42753163, -0.19258936],
       [ 0.0466683 , -0.16183038,  0.61364186]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.5442238 ,  0.22780192,  0.704233  ], dtype=float32)>


In [18]:
"""
tf.nn.softmax computes softmax activations
softmax = exp(logits)/reduce_sum(expt(logits),dim)
"""
def hypothesis(X) :
  return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.2120243e-02 9.1549844e-01 7.2381392e-02]
 [9.2492111e-02 2.1506955e-01 6.9243836e-01]
 [7.2892822e-02 4.3151308e-02 8.8395578e-01]
 [5.1223580e-02 9.7793126e-03 9.3899715e-01]
 [2.2841135e-07 9.9990880e-01 9.0914138e-05]
 [3.2851368e-03 9.2520788e-02 9.0419406e-01]
 [2.9006392e-06 9.9783212e-01 2.1650069e-03]
 [3.3683312e-07 9.9931419e-01 6.8550062e-04]], shape=(8, 3), dtype=float32)


In [24]:
#softmax test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.7174977  0.03722719 0.24527511]], shape=(1, 3), dtype=float32)


In [26]:
# Cost function : cross entropy
def cost_fn(X , Y) :
  logits = hypothesis(X)
  cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
  cost_mean = tf.reduce_mean(cost)
  return cost_mean

print(cost_fn(x_data , y_data))

tf.Tensor(4.722356, shape=(), dtype=float32)


In [27]:
#Gradient Function
def grad_fn(X , Y):
  with tf.GradientTape() as tape:
    loss = cost_fn(X , Y)
    grads = tape.gradient(loss , variables)
    return grads

print(grad_fn(x_data , y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.17200428, -0.1745269 ,  0.34653118],
       [-1.59407   ,  1.5331991 ,  0.06087089],
       [-1.527395  ,  0.64792466,  0.8794704 ],
       [-1.5294344 ,  0.513     ,  1.0164343 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.22099784,  0.15913431,  0.06186352], dtype=float32)>]


In [29]:
#Train
def fit(X, Y, epochs=2000 , verbose = 100):
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
  for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 3.609033
Loss at epoch 100: 0.679015
Loss at epoch 200: 0.597252
Loss at epoch 300: 0.540765
Loss at epoch 400: 0.490731
Loss at epoch 500: 0.443269
Loss at epoch 600: 0.396927
Loss at epoch 700: 0.351063
Loss at epoch 800: 0.305701
Loss at epoch 900: 0.263351
Loss at epoch 1000: 0.238793
Loss at epoch 1100: 0.226854
Loss at epoch 1200: 0.216104
Loss at epoch 1300: 0.206297
Loss at epoch 1400: 0.197314
Loss at epoch 1500: 0.189055
Loss at epoch 1600: 0.181435
Loss at epoch 1700: 0.174385
Loss at epoch 1800: 0.167843
Loss at epoch 1900: 0.161757
Loss at epoch 2000: 0.156082


In [30]:
#Prediction Check

sample_data = [[2, 1, 3, 2]] # answer label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a,1))  # index : 2

tf.Tensor([[0.00157137 0.08036441 0.91806424]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [31]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b,1))
print(tf.argmax(y_data,1))  #matches with y_data

tf.Tensor(
[[2.83591316e-06 1.30188034e-03 9.98695314e-01]
 [1.57136726e-03 8.03644136e-02 9.18064237e-01]
 [1.44851782e-07 1.61430359e-01 8.38569462e-01]
 [4.64985897e-06 8.52740943e-01 1.47254378e-01]
 [2.56197989e-01 7.31875956e-01 1.19259795e-02]
 [1.34828150e-01 8.65119934e-01 5.18369197e-05]
 [7.49832153e-01 2.50122398e-01 4.54276415e-05]
 [9.21672821e-01 7.83264562e-02 7.90108118e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


Convert as Class

In [32]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.347447
Loss at epoch 500: 0.448661
Loss at epoch 1000: 0.273383
Loss at epoch 1500: 0.198863
Loss at epoch 2000: 0.163236
